In [1]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale

In [2]:
pd.set_option('display.max_rows', 100)  # Disable row limit
pd.set_option('display.max_columns', 40)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
# Set the display precision to 2 decimal places  
pd.set_option('display.precision', 2)  
  
# Or set the float format to always show 2 decimal places  
pd.set_option('display.float_format', '{:.2f}'.format)  


In [3]:
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"


In [24]:
df = pd.read_json("output/20241212_181140_fast_real-estate-to-rent_real-estate-for-sale.jsonl", lines=True)
df

,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,...,Bedrooms,Bathrooms,images,description,Floor,Postal code,Square meter price,Condition,Share,Density,Coverage,Land type,Plot Type,Parcel number,Planning zone,Area,Registration block,Registration number,Reference number,Minimum stay
0,https://www.bazaraki.com/adv/5562058_4-bedroom-villa-to-rent/,4-bedroom villa to rent,1450,NaN,NaN,"Famagusta, Protaras",07.12.2024 07:46,5562058,NaN,363,NaN,NaN,False,Cyprus real estate to rent,Houses to rent,145.00,Allowed,Villa,Uncovered,400.00,...,4.00,2,"[https://cdn1.bazaraki.com/media/cache1/c4/3e/c43e5334d9e92ccf7d4319b2fbb1a962.webp, https://cdn...","The property is 4-bedroom villa with a total internal covered area of 145sqm in Pernera, Protara...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.bazaraki.com/adv/5555117_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,4500,NaN,NaN,"Limassol, Limassol - Neapolis",06.12.2024 11:12,5555117,NaN,219,34.69,33.06,False,Cyprus real estate to rent,"Apartments, flats to rent",187.00,Not allowed,Apartment,Covered,NaN,...,3.00,3,"[https://cdn1.bazaraki.com/media/cache1/2a/73/2a73d4813920064415ce92d4f448f1f4.webp, https://cdn...","The premium complex is located in the heart of the popular area of Neapolis, where the main attr...",4th,3106.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.bazaraki.com/adv/5508708_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1300,1600.00,NaN,"Limassol, Mouttagiaka Tourist Area",05.12.2024 14:38,5508708,NaN,454,34.70,33.11,False,Cyprus real estate to rent,"Apartments, flats to rent",70.00,Not allowed,Apartment,Uncovered,NaN,...,2.00,2,"[https://cdn1.bazaraki.com/media/cache1/95/6c/956c51d4a9597be8875ea1801a3854e4.webp, https://cdn...",2 Bedroom seafront apartment,1st,4531.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.bazaraki.com/adv/5526092_1-bedroom-apartment-to-rent/,1-bedroom apartment to rent,1450,NaN,NaN,"Limassol, Germasogeia Tourist Area",05.12.2024 08:44,5526092,NaN,501,34.70,33.09,False,Cyprus real estate to rent,"Apartments, flats to rent",50.00,Allowed,Apartment,Uncovered,NaN,...,1.00,1,"[https://cdn1.bazaraki.com/media/cache1/50/28/5028a1e5064d48a9e3accfb67a15d490.webp, https://cdn...","Beautiful 1 Bedroom apartment, fully furnished and appliances, 30M garden, private parking, exce...",Ground floor,4046.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.bazaraki.com/adv/5541691_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1700,NaN,NaN,"Limassol, Limassol - Neapolis",08.12.2024 14:37,5541691,NaN,370,34.69,33.07,False,Cyprus real estate to rent,"Apartments, flats to rent",80.00,Not allowed,Apartment,Uncovered,NaN,...,2.00,1,"[https://cdn1.bazaraki.com/media/cache1/a5/67/a56719f143a40c98733bf63c7bc77d60.webp, https://cdn...",Rent is possible for a period of 4 to 10 months.\n-The renovation has just been completed. All f...,1st,3107.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,https://www.bazaraki.com/adv/5064550_1-bedroom-apartment-to-rent/,Spacious 1 bedroom 65 sqm beachfront apartment,59,79.00,NaN,"Limassol, Limassol - Neapolis",Yesterday 19:08,5064550,NaN,717,NaN,NaN,False,Cyprus real estate to rent,Short term,59.00,Not allowed,"Apartment, flat",NaN,NaN,...,1.00,NaN,"[https://cdn1.bazaraki.com/media/cache1/c5/74/c5746727ae9aedee718ff4a71fe651d0.webp, https://cdn...","Spacious 1 bedroom 65 Sqm beachfront apartment, which has a sliding door to separate the living ...",NaN,4532.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 days
116,https://www.bazaraki.com/adv/5546021_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,6500,N

In [21]:
df.query("cat0.str.contains('sale', case=False, na=False) and cat1.str.contains('flats', case=False, na=False)")

,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,description,images,sold,cat0,cat1,Property area,Pets,Type,...,Air conditioning,Construction year,Energy Efficiency,Bedrooms,Bathrooms,Floor,Square meter price,Condition,Share,Density,Coverage,Land type,Plot Type,Parcel number,Planning zone,Area,Registration block,Registration number,Reference number,Minimum stay
16,https://www.bazaraki.com/adv/5260850_3-bedroom-penthouse-for-sale/,3-bedroom penthouse fоr sаle,525000,NaN,2419.00,"Nicosia, Egkomi",13.11.2024 10:01,5260850,NaN,1918,35.17,33.33,"La Maison Residencies in Engomi, it’s really unique development ready to satisfy the higher expe...","[https://cdn1.bazaraki.com/media/cache1/49/3a/493acfa313681dad9dc3506cdc3cd553.webp, https://cdn...",False,Cyprus real estate for sale,"Apartments, flats for sale",217.00,NaN,Penthouse,...,"Full, all rooms",2024,A,3.00,4,4th,€2.419 /,Brand new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,https://www.bazaraki.com/adv/5503436_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,376000,NaN,3082.00,"Paphos, Paphos - Agios Theodoros",20.11.2024 10:09,5503436,NaN,309,34.77,32.42,2 and 3 bedrooms，communal GYM，rooftop swimming pool， underfloor heating,"[https://cdn1.bazaraki.com/media/cache1/89/9a/899aeafa91fac401f6ba6590d7cf0c15.webp, https://cdn...",False,Cyprus real estate for sale,"Apartments, flats for sale",122.00,NaN,Apartment,...,"Full, all rooms",NaN,In Progress,2.00,2,7th,€3.082 /,Brand new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,https://www.bazaraki.com/adv/5433785_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,480000,NaN,4034.00,"Limassol, Agios Athanasios",06.12.2024 10:28,5433785,NaN,129,34.71,33.06,Sea Breeze 5 - Agios Athanasios\nConstruction Status – Under Construction\nLuxury 2 Bedroom Apar...,"[https://cdn1.bazaraki.com/media/cache1/48/96/48965f146eec15aef11039592acbeb96.webp, https://cdn...",False,Cyprus real estate for sale,"Apartments, flats for sale",119.00,NaN,Apartment,...,"Full, all rooms",NaN,A,2.00,2,1st,€4.034 /,Under construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,https://www.bazaraki.com/adv/5321953_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,178000,NaN,1548.00,"Nicosia, Nicosia - Kaimakli",06.12.2024 12:00,5321953,NaN,658,35.19,33.38,"Brand new 3-bedroom apartment in Kaimakli, Nicosia\nNo VAT applies.\nSubsidy up to €65,000 appli...","[https://cdn1.bazaraki.com/media/cache1/8f/aa/8faa9e576f91e6c849fffc575a3b4fb8.webp, https://cdn...",False,Cyprus real estate for sale,"Apartments, flats for sale",115.00,NaN,Apartment,...,No,2019,B,3.00,2,3rd,€1.548 /,Brand new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,3.00,NaN,NaN
29,https://www.bazaraki.com/adv/5326549_2-bedroom-penthouse-for-sale/,2-bedroom penthouse fоr sаle,310000,NaN,3780.00,"Larnaca, Larnaka - Sotiros",06.12.2024 16:33,5326549,NaN,327,34.94,33.63,2 bedroom Luxury brand new Penthouse with Roof Garden\n>\tREADY TO MOVE IN and just delivered\n>...,"[https://cdn1.bazaraki.com/media/cache1/c5/ce/c5ceb83a22f0fff9b25f0fe163c411e6.webp, https://cdn...",False,Cyprus real estate for sale,"Apartments, flats for sale",82.00,NaN,Penthouse,...,No,2024,A,2.00,2,4th,€3.780 /,Brand new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,https://www.bazaraki.com/adv/4763357_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,657000,NaN,7640.00,"Limassol, Agios Tychon",06.12.2024 16:36,4763357,NaN,39,34.72,33.15,Project a quiet hillside on the eastern side of Limassol.\nIts prime location on an elevated pla...,"[https://cdn1.bazaraki.com/media/cache1/ef/2f/ef2fc52aa26f36b349102c09a0f68654.webp, https://cdn...",False,Cyprus real estate for sale,"Apartments, flats for sale",86.00,NaN,Apartment,...,NaN,2024,A,2.00,2,NaN,€7.640 /,Under construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1964,NaN
31,https://www.bazaraki.com/adv/5511877_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,650000,NaN,11404.00,"Limass

In [16]:
df[["ad_id", "posted", "url"]].sort_values("posted", ascending=False)


,ad_id,posted,url
76,5508233,Yesterday 21:02,https://www.bazaraki.com/adv/5508233_2-bedroom-apartment-to-rent/
66,5568500,Yesterday 20:58,https://www.bazaraki.com/adv/5568500_2-bedroom-apartment-for-sale/
77,5415213,Yesterday 20:57,https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/
15,5532378,Yesterday 20:32,https://www.bazaraki.com/adv/5532378_3-bedroom-detached-house-for-sale/
65,5370624,Yesterday 20:26,https://www.bazaraki.com/adv/5370624_2-bedroom-apartment-for-sale/
...,...,...,...
14,5520154,04.12.2024 14:25,https://www.bazaraki.com/adv/5520154_3-bedroom-villa-to-rent/
10,5233452,03.12.2024 09:24,https://www.bazaraki.com/adv/5233452_serviced-office/
21,5432121,03.12.2024 08:23,https://www.bazaraki.com/adv/5432121_residential-land-768-m2/
17,4535668,01.12.2024 09:09,https://www.bazaraki.com/adv/4535668_agricultural-land-11391-m2/


In [7]:
df.ad_id.describe()

count      4481.00
mean    5477942.25
std      143431.79
min     2886930.00
25%     5465868.00
50%     5524599.00
75%     5548284.00
max     5562527.00
Name: ad_id, dtype: float64

In [5]:
df.ad_id.hist().show()

In [42]:
df[["city", "district"]] = df.location.str.split(",", expand=True)

In [43]:

df.price_per_sqm = df.price / (df["Property area"].str.replace("m²", "").astype(int)).round(2)


In [44]:
df.iloc[0].T

url                                                    https://www.bazaraki.com/adv/5498789_2-bedroom-apartment-to-rent/
title                                                                                        2-bedroom apartment to rent
price                                                                                                               3100
price_per_sqm                                                                                                      31.00
location                                                                                          Limassol, Agios Tychon
posted                                                                                                  03.12.2024 11:01
ad_id                                                                                                            5498789
reference_number                                                                                                     NaN
views                           

In [ ]:
selected_df = saledf[saledf.price < 5000]
print(f"removed {len(saledf) - len(selected_df)} rows")
df = selected_df

removed 12671 rows


In [ ]:
# selected_df = df.query("`Bedrooms` in ('3', '2', '1', 'Studio')")
# print(f"removed {len(df) - len(selected_df)} rows")
# df = selected_df

removed 0 rows


In [32]:
saledf = pd.read_json("output/_real-estate-for-sale_apartments-flats__20241207_133456.jsonl", lines=True)
saledf[["city", "district"]] = saledf.location.str.split(",", expand=True)
saledf

,url,title,price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,description,images,Property area,Type,Floor,Parking,Condition,Furnishing,Included,Air conditioning,Energy Efficiency,Bedrooms,Square meter price,Bathrooms,Reference number,Postal code,Construction year,Online viewing,Registration block,Registration number,city,district
0,https://www.bazaraki.com/adv/5270074_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,110000,2558.00,"Famagusta, Kapparis",Yesterday 16:43,5270074,NaN,1313,NaN,NaN,"A 1-bedroom Second Floor Apartment in Kapparis, Famagusta. The Apartment is located at the top f...","[https://cdn1.bazaraki.com/media/cache1/16/41/1641c407b2fd66410b3a69c9c71fcfc4.webp, https://cdn...",43 m²,Apartment,2nd,Covered,Resale,Fully Furnished,"Pool, Balcony, Storage room","Full, all rooms",N/A,1,€2.558 /m²,1,NaN,NaN,NaN,NaN,NaN,NaN,Famagusta,Kapparis
1,https://www.bazaraki.com/adv/5513457_3-bedroom-penthouse-for-sale/,3-bedroom penthouse fоr sаle,550000,4583.00,"Limassol, Agios Athanasios",03.12.2024 23:05,5513457,NaN,5,NaN,NaN,"Reference ID: #SA34867Price details: €550,000 +VATA new project for a three-floor residential bu...","[https://cdn1.bazaraki.com/media/cache1/e7/b0/e7b0e9e3b0ffc1ead147229f44bafbcd.webp, https://cdn...",120 m²,Penthouse,3rd,Covered,Brand new,Unfurnished,"Storage room, Balcony",No,N/A,3,€4.583 /m²,2,SA34867,NaN,NaN,NaN,NaN,NaN,Limassol,Agios Athanasios
2,https://www.bazaraki.com/adv/5225611_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,600000,5085.00,"Limassol, Limassol - Neapolis",Yesterday 16:42,5225611,NaN,23,34.68,33.03,"Luxury Sea view apartment, with panoramic sea views, within walking distance to the beach and th...","[https://cdn1.bazaraki.com/media/cache1/84/2f/842f66392ea01b68c17398a3b9967564.webp, https://cdn...",118 m²,Apartment,NaN,No,Resale,NaN,NaN,NaN,B+,3,€5.085 /m²,4,7444,3030.00,2023,NaN,NaN,NaN,Limassol,Limassol - Neapolis
3,https://www.bazaraki.com/adv/5294642_studio-apartment-for-sale/,Studio apartment fоr sаle,197000,6567.00,"Limassol, Historical Center",04.12.2024 08:43,5294642,NaN,7,34.68,33.04,This project features 25 serviced studio apartments located in Limassol's Old Town.\nAmenities i...,"[https://cdn1.bazaraki.com/media/cache1/0f/d6/0fd68165844de93bfca2214d0604c3e5.webp, https://cdn...",30 m²,Apartment,3rd,No,Brand new,NaN,Elevator,"Full, all rooms",A,Studio,€6.567 /m²,NaN,52865,NaN,NaN,No,NaN,NaN,Limassol,Historical Center
4,https://www.bazaraki.com/adv/5556626_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,169500,1863.00,"Paphos, Geroskipou",03.12.2024 15:01,5556626,NaN,37,34.76,32.46,This spacious first floor fully furnished apartment is in a popular complex overlooking a large ...,"[https://cdn1.bazaraki.com/media/cache1/a3/bc/a3bc868e178410266c6a166a70084d4b.webp, https://cdn...",91 m²,Apartment,NaN,NaN,Resale,NaN,NaN,NaN,N/A,2,€1.863 /m²,1,MDVGPH2,NaN,NaN,NaN,NaN,NaN,Paphos,Geroskipou
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12666,https://www.bazaraki.com/adv/5513325_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,252000,4000.00,"Paphos, Tombs Of the Kings",27.11.2024 19:35,5513325,NaN,68,34.77,32.40,"YL Luxury Homes Ltd presents a new luxury and modern development in the heart of Kato Pafos, clo...","[https://cdn1.bazaraki.com/media/cache1/a5/f2/a5f2df0c92055456183edc04b3265cbd.webp, https://cdn...",63 m²,Apartment,1st,Covered,Under construction,NaN,"Pool, Elevator, Balcony, Storage room",NaN,A,1,€4.000 /m²,1,NaN,NaN,NaN,NaN,NaN,NaN,Paphos,Tombs Of the Kings
12667,https://www.bazaraki.com/adv/5493768_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,215000,2312.00,"Nicosia, Stelmek Area",28.11.2024 10:00,5493768,NaN,363,35.14,33.30,"Modern Living in Stelmek, Nicosia – Limited Apartments Available!\nBe part of this exciting new ...","[https://cdn1.bazaraki.com/media/cache1/85/de/85de8c5d2727aef312fc

In [67]:
salestatdf = saledf.query("Condition == 'Resale'").groupby(["city", "Bedrooms"]).price_per_sqm.describe()
rentstatdf = df.groupby(["city", "Bedrooms"]).price_per_sqm.describe()
common_index = salestatdf.index.intersection(rentstatdf.index)

result = rentstatdf.loc[common_index] / salestatdf.loc[common_index] * 12 * 100
pd.concat([result, rentstatdf, salestatdf], axis=1, keys=['result', 'rent', 'sale'])

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract



result  \
                       count   
city      Bedrooms             
Famagusta 1           518.92   
          2           364.71   
          3           200.00   
          Studio     1600.00   
Larnaca   1          1712.00   
          2          1555.84   
          3           655.67   
          4           857.14   
          Studio     1950.00   
Limassol  1          4015.84   
          2          2166.37   
          3          1730.72   
          4          1214.46   
          5           600.00   
          6 and more 1200.00   
          Studio     9600.00   
Nicosia   1          5785.71   
          2          2488.56   
          3          1901.69   
          4          1800.00   
          5           600.00   
          Studio     4971.43   
Paphos    1          1440.00   
          2          1708.61   
          3          1552.94   
          4          1200.00   
          Studio     2100.00   
Famagusta 4              NaN   
          5              NaN   
Larnaca   6 and more     NaN   
Nicosia   6 and more     NaN   
Paphos    6 and more     NaN   

                            \
                      mean   
city      Bedrooms           
Famagusta 1           3.81   
          2           4.03   
          3           5.20   
          Studio      4.80   
Larnaca   1           6.59   
          2            inf   
          3           6.18   
          4           8.21   
          Studio       inf   
Limassol  1            inf   
          2            inf   
          3            inf   
          4           8.93   
          5           4.39   
          6 and more 33.25   
          Studio      6.33   
Nicosia   1           7.46   
          2            inf   
          3           5.82   
          4           5.11   
          5           1.71   
          Studio      8.01   
Paphos    1           5.85   
          2            inf   
          3           4.78   
          4           4.58   
          Studio      8.22   
Famagusta 4            NaN   
          5            NaN   
Larnaca   6 and more   NaN   
Nicosia   6 and more   NaN   
Paphos    6 and more   NaN   

                             \
                        std   
city      Bedrooms            
Famagusta 1            2.19   
          2            1.73   
          3            4.84   
          Studio       2.11   
Larnaca   1            5.33   
          2             NaN   
          3            3.72   
          4            9.84   
          Studio        NaN   
Limassol  1             NaN   
          2             NaN   
          3             NaN   
          4           62.36   
          5            1.54   
          6 and more 181.88   
          Studio      17.23   
Nicosia   1           81.71   
          2             NaN   
          3            4.73   
          4            3.29   
          5             NaN   
          Studio       8.82   
Paphos    1            6.55   
          2             NaN   
          3            2.71   
          4            1.91   
          Studio      11.90   
Famagusta 4             NaN   
          5             NaN   
Larnaca   6 and more    NaN   
Nicosia   6 and more    NaN   
Paphos    6 and more    NaN   

                            \
                       min   
city      Bedrooms           
Famagusta 1           5.42   
          2           7.98   
          3           8.18   
          Studio      7.37   
Larnaca   1           0.10   
          2           2.70   
          3           8.68   
          4           8.74   
          Studio     10.39   
Limassol  1          32.61   
          2           5.09   
          3          28.70   
          4           4.63   
          5           7.16   
          6 and more  3.86   
          Studio      0.09   
Nicosia   1           4.42   
          2           6.38   
          3           6.77   
          4           6.23   
          5           2.08   
          Studio      5.08   
Paphos    1   

In [24]:
df.query("Bedrooms == '2' and city=='Limassol'").price_per_sqm.hist(bins=100)

In [ ]:
px.histogram(df.query("Bedrooms == '1' and price_per_sqm < 200"), x="price_per_sqm", nbins=100, color="city", barmode="overlay").show()


In [19]:
px.histogram(df, x="price", barmode='overlay', facet_col="city", color="Bedrooms").show()


In [47]:
df.sort_values("views", ascending=False)


,url,title,price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,description,images,Property area,Pets,Type,Floor,Parking,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,Postal code,Construction year,Reference number,city,district
1018,https://www.bazaraki.com/adv/4025924_1-bedroom-apartment-to-rent/,Studio apartment to rent,600,17.14,"Limassol, Trimiklini",18.11.2024 13:34,4025924,NaN,27225,34.85,32.91,Brand new large studios for rent.\nThe studios are fully furnished. In the price is included ele...,"[https://cdn1.bazaraki.com/media/cache1/82/90/829044a36d68bee9ea901a0c162dd458.webp, https://cdn...",35 m²,Not allowed,Apartment,NaN,Covered,Fully Furnished,"Garden, Balcony",No,"Full, all rooms",N/A,Studio,1,4730.00,2023,NaN,Limassol,Trimiklini
4211,https://www.bazaraki.com/adv/4852027_studio-apartment-to-rent/,Studio apartment to rent,450,inf,"Larnaca, Perivolia Larnakas",13:30,4852027,NaN,14326,NaN,NaN,𝐍𝐄𝐖 𝐒𝐓𝐔𝐃𝐈𝐎 𝐀𝐏𝐀𝐑𝐓𝐌𝐄𝐍𝐓𝐒 𝐅𝐎𝐑 𝐑𝐄𝐍𝐓\nFully Furnished\nAircondition\nSmart TVs\nWi-Fi internet\nWashin...,"[https://cdn1.bazaraki.com/media/cache1/bb/e8/bbe8dae73c9864bc080fead856397954.webp, https://cdn...",0 m²,Not allowed,Apartment,1st,Covered,Fully Furnished,NaN,Yes,"Full, all rooms",A,Studio,1,7560.00,2023,NaN,Larnaca,Perivolia Larnakas
425,https://www.bazaraki.com/adv/4627575_2-bedroom-apartment-to-rent/,1-bedroom apartment to rent,800,16.00,"Limassol, Germasogeia Tourist Area",30.11.2024 22:24,4627575,NaN,5291,NaN,NaN,Experience Limassol from the best area close to the beach . Calls after 10.00. only serious per...,"[https://cdn1.bazaraki.com/media/cache1/50/7d/507dc16d8a2b10759bb0034d7724adef.webp, https://cdn...",50 m²,Not allowed,Apartment,2nd,No,Fully Furnished,"Elevator, Alarm, Balcony",No,Partly,C,1,1,4040.00,Older,NaN,Limassol,Germasogeia Tourist Area
3929,https://www.bazaraki.com/adv/5256064_studio-apartment-to-rent/,Studio apartment to rent,300,4.62,"Nicosia, Klirou",03.12.2024 12:11,5256064,NaN,5251,NaN,NaN,"Studio for rent in Klirou, fully furnished, preferred people from Europe who work in Cyprus.\nΕν...","[https://cdn1.bazaraki.com/media/cache1/aa/d8/aad85bf8dccca73093c28cc66a1bd898.webp, https://cdn...",65 m²,Not allowed,Apartment,Ground floor,NaN,Fully Furnished,NaN,NaN,"Full, all rooms",B,Studio,1,NaN,NaN,NaN,Nicosia,Klirou
2938,https://www.bazaraki.com/adv/4776408_3-bedroom-apartment-to-rent/,2-bedroom apartment to rent,700,7.61,"Larnaca, Livadia Larnakas",19.11.2024 06:41,4776408,NaN,4999,NaN,NaN,Ενοικιαζεται διαμερισμα στη Λαρνακα. Ενοικιαζεται ΜΟΝΟ για γυναικες ή ενα ζευγαρι.Διαθετει 2 δωμ...,"[https://cdn1.bazaraki.com/media/cache1/f9/27/f9272266c5e7a12b27bd6fd3555cd05b.webp, https://cdn...",92 m²,Not allowed,Apartment,2nd,Uncovered,Fully Furnished,Balcony,No,"Full, all rooms",A,2,2,6052.00,2011,NaN,Larnaca,Livadia Larnakas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4288,https://www.bazaraki.com/adv/5548602_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,2300,23.47,"Limassol, Limassol - Tsirion",09:13,5548602,NaN,1,34.70,33.02,"New apartment, built in 2023. 2 bedrooms. Fully furnished.","[https://cdn1.bazaraki.com/media/cache1/af/f5/aff5abdbc5d43cae36407a5e7f45780c.webp, https://cdn...",98 m²,Not allowed,Apartment,2nd,Covered,Fully Furnished,"Elevator, Balcony",No,"Full, all rooms",A,2,1,3081.00,2023,NaN,Limassol,Limassol - Tsirion
4228,https://www.bazaraki.com/adv/5519958_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,3000,16.67,"Nicosia, Egkomi",14:40,5519958,NaN,1,35.16,33.33,Rent this beautiful Flat through www.rentonline.today and save 30% on your first monthly rent!. ...,"[https://cdn1.bazaraki.com/media/cache1/72/bb/72bb10111004854bce49ff4443d641a1.webp, https://cdn...",180 m²,Not allowed,Apartment,Ground floor,Covered,Fully Furnished,NaN,NaN,"Full, all rooms",A,3,3,NaN,2023,rentonline-94,Nicosia,Egkomi
4229,https://www.bazaraki.com/adv/54755

In [ ]:
saledf = pd.read_json("output/_real-estate-for-sale_apartments-flats__20241207_133456.jsonl", lines=True)
saledf

,url,title,price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,description,images,Property area,Type,Floor,Parking,Condition,Furnishing,Included,Air conditioning,Energy Efficiency,Bedrooms,Square meter price,Bathrooms,Reference number,Postal code,Construction year,Online viewing,Registration block,Registration number
0,https://www.bazaraki.com/adv/5270074_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,110000,2558.0,"Famagusta, Kapparis",Yesterday 16:43,5270074,NaN,1313,NaN,NaN,"A 1-bedroom Second Floor Apartment in Kapparis, Famagusta. The Apartment is located at the top f...","[https://cdn1.bazaraki.com/media/cache1/16/41/1641c407b2fd66410b3a69c9c71fcfc4.webp, https://cdn...",43 m²,Apartment,2nd,Covered,Resale,Fully Furnished,"Pool, Balcony, Storage room","Full, all rooms",N/A,1,€2.558 /m²,1,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.bazaraki.com/adv/5513457_3-bedroom-penthouse-for-sale/,3-bedroom penthouse fоr sаle,550000,4583.0,"Limassol, Agios Athanasios",03.12.2024 23:05,5513457,NaN,5,NaN,NaN,"Reference ID: #SA34867Price details: €550,000 +VATA new project for a three-floor residential bu...","[https://cdn1.bazaraki.com/media/cache1/e7/b0/e7b0e9e3b0ffc1ead147229f44bafbcd.webp, https://cdn...",120 m²,Penthouse,3rd,Covered,Brand new,Unfurnished,"Storage room, Balcony",No,N/A,3,€4.583 /m²,2,SA34867,NaN,NaN,NaN,NaN,NaN
2,https://www.bazaraki.com/adv/5225611_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,600000,5085.0,"Limassol, Limassol - Neapolis",Yesterday 16:42,5225611,NaN,23,34.684042,33.031313,"Luxury Sea view apartment, with panoramic sea views, within walking distance to the beach and th...","[https://cdn1.bazaraki.com/media/cache1/84/2f/842f66392ea01b68c17398a3b9967564.webp, https://cdn...",118 m²,Apartment,NaN,No,Resale,NaN,NaN,NaN,B+,3,€5.085 /m²,4,7444,3030.0,2023,NaN,NaN,NaN
3,https://www.bazaraki.com/adv/5294642_studio-apartment-for-sale/,Studio apartment fоr sаle,197000,6567.0,"Limassol, Historical Center",04.12.2024 08:43,5294642,NaN,7,34.679846,33.044669,This project features 25 serviced studio apartments located in Limassol's Old Town.\nAmenities i...,"[https://cdn1.bazaraki.com/media/cache1/0f/d6/0fd68165844de93bfca2214d0604c3e5.webp, https://cdn...",30 m²,Apartment,3rd,No,Brand new,NaN,Elevator,"Full, all rooms",A,Studio,€6.567 /m²,NaN,52865,NaN,NaN,No,NaN,NaN
4,https://www.bazaraki.com/adv/5556626_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,169500,1863.0,"Paphos, Geroskipou",03.12.2024 15:01,5556626,NaN,37,34.761539,32.458060,This spacious first floor fully furnished apartment is in a popular complex overlooking a large ...,"[https://cdn1.bazaraki.com/media/cache1/a3/bc/a3bc868e178410266c6a166a70084d4b.webp, https://cdn...",91 m²,Apartment,NaN,NaN,Resale,NaN,NaN,NaN,N/A,2,€1.863 /m²,1,MDVGPH2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12666,https://www.bazaraki.com/adv/5513325_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,252000,4000.0,"Paphos, Tombs Of the Kings",27.11.2024 19:35,5513325,NaN,68,34.774553,32.404895,"YL Luxury Homes Ltd presents a new luxury and modern development in the heart of Kato Pafos, clo...","[https://cdn1.bazaraki.com/media/cache1/a5/f2/a5f2df0c92055456183edc04b3265cbd.webp, https://cdn...",63 m²,Apartment,1st,Covered,Under construction,NaN,"Pool, Elevator, Balcony, Storage room",NaN,A,1,€4.000 /m²,1,NaN,NaN,NaN,NaN,NaN,NaN
12667,https://www.bazaraki.com/adv/5493768_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,215000,2312.0,"Nicosia, Stelmek Area",28.11.2024 10:00,5493768,NaN,363,35.140516,33.300250,"Modern Living in Stelmek, Nicosia – Limited Apartments Available!\nBe part of this exciting new ...","[https://cdn1.bazaraki.com/media/cache1/85/de/85de8c5d2727aef312fc7f49a7b028ca.webp, https://cdn...",93 m²,Apartment,NaN,Covered,Brand new,NaN,"Garden, Elevator, Balcony, Storage room",NaN,A,2,€2.312 /

In [ ]:
saledf.iloc[0].T

url                                                     https://www.bazaraki.com/adv/5270074_1-bedroom-apartment-for-sale/
title                                                                                         1-bedroom apartment fоr sаle
price                                                                                                               110000
price_per_sqm                                                                                                       2558.0
location                                                                                               Famagusta, Kapparis
posted                                                                                                     Yesterday 16:43
ad_id                                                                                                              5270074
reference_number                                                                                                       NaN
views           

In [ ]:
saledf[["city", "district"]] = saledf.location.str.split(",", expand=True)